# Optimisation contrainte avec multiplicateurs de Lagrange et autograd

L'optimisation contrainte est courante dans la résolution de problèmes d'ingénierie. Un exemple prototypique (de Greenberg, Advanced Engineering Mathematics, Ch 13.7) est de trouver le point sur un plan qui est le plus proche de l'origine. 

Le plan est défini par l'équation 2x−y+z=3, et on cherche à minimiser x2+y2+z2 sous la contrainte d'égalité définie par le plan. scipy.optimize.minimize fournit une interface assez pratique pour résoudre un problème comme celui-ci, et illustré ici.

Lien : https://kitchingroup.cheme.cmu.edu/blog/2018/11/03/Constrained-optimization-with-Lagrange-multipliers-and-autograd/

In [13]:
import numpy as np
from scipy.optimize import minimize

def objective(X):
    x, y, z = X
    return x**2 + y**2 + z**2

def eq(X):
    x, y, z = X
    return 2 * x - y + z - 3

sol = minimize(objective, [1, -0.5, 0.5], constraints={'type': 'eq', 'fun': eq})
sol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.5
       x: [ 1.000e+00 -5.000e-01  5.000e-01]
     nit: 1
     jac: [ 2.000e+00 -1.000e+00  1.000e+00]
    nfev: 4
    njev: 1

J'aime beaucoup la fonction de minimisation, bien que je ne sois pas fou de la façon dont les contraintes sont fournies. L'alternative était qu'il y avait un argument pour les contraintes d'égalité et un autre pour les contraintes d'inégalité. Analogues aux fonctions d'événement scipy.integrate.solve_ivp, elles auraient également pu utiliser des attributs de fonction.

Parfois, il peut être souhaitable de revenir à l'essentiel, surtout si vous n'êtes pas au courant de la fonction de minimisation ou si vous soupçonnez qu'elle ne fonctionne pas correctement et que vous souhaitez une réponse indépendante. 
Ensuite, nous regardons comment construire ce problème d'optimisation sous contrainte en utilisant des multiplicateurs de Lagrange. Cela convertit le problème en un problème d'optimisation augmenté sans contrainte sur lequel nous pouvons utiliser fsolve. L'essentiel de cette méthode est que nous formulons un nouveau problème :

F(X)=f(X)−λg(X)

puis résolvez les équations résultantes simultanées :

Fx(X)=Fy(X)=Fz(X)=g(X)=0
où Fx est la dérivée de f∗ par rapport à x, et g(X) est la contrainte d'égalité écrite telle qu'elle soit égale à zéro. 

Puisque nous nous retrouvons avec quatre équations égales à zéro, nous pouvons simplement utiliser fsolve pour obtenir la solution. Il y a de nombreuses années, j'ai utilisé une approximation aux différences finies des dérivées. Aujourd'hui, nous utilisons autograd pour obtenir les dérivées souhaitées. C'est ici.



In [14]:
import autograd.numpy as np
from autograd import grad

def F(L):
    'Augmented Lagrange function'
    x, y, z, _lambda = L
    return objective([x, y, z]) - _lambda * eq([x, y, z])

# Gradients of the Lagrange function
dfdL = grad(F, 0)

# Find L that returns all zeros in this function.
def obj(L):
    x, y, z, _lambda = L
    dFdx, dFdy, dFdz, dFdlam = dfdL(L)
    return [dFdx, dFdy, dFdz, eq([x, y, z])]

from scipy.optimize import fsolve
x, y, z, _lam = fsolve(obj, [0.0, 0.0, 0.0, 1.0])
print(f'The answer is at {x, y, z}')

The answer is at (1.0, -0.5, 0.5)


C'est la même réponse que précédemment. Notez que nous nous sommes toujours appuyés sur un solveur de boîte noire à l'intérieur de fsolve (au lieu de minimiser à l'intérieur), mais il pourrait être plus clair quel problème nous résolvons (par exemple, trouver des zéros). Cela demande un peu plus de travail pour configurer cela, car nous devons construire la fonction augmentée, mais autograd rend assez pratique la configuration de la fonction objectif finale que nous voulons résoudre.

Comment savons-nous que nous sommes au minimum ? Nous pouvons vérifier que le hessien est défini positif dans la fonction d'origine que nous voulions minimiser. Vous pouvez voir ici que le tableau est défini positif, par ex. toutes les valeurs propres sont positives. autograd rend cela facile aussi.

In [15]:
from autograd import hessian
h = hessian(objective, 0)
h(np.array([x, y, z]))

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [16]:
# Dans le cas où il ne ressort pas de cette structure que les valeurs propres sont toutes positives, nous les calculons ici :

np.linalg.eig(h(np.array([x, y, z])))[0]

array([2., 2., 2.])

 In summary, autograd continues to enable advanced engineering problems to be solved.

Copyright (C) 2018 by John Kitchin. See the License for information about copying.